# Network IN Network

因为NIN的作者主要拿`Maxout Network`进行比较，因此我们首先有必要搞懂这个网络。

## Maxout Network

该网络是Goodfellow于2013年在ICML上贡献的一篇论文，Maxout于Dropout有很多相似的地方。具体的Maxout论文笔记请参考[这篇博文](http://blog.csdn.net/hjimce/article/details/50414467)

Maxout主要就是对原先传统的MLP层做改进，将原来计算隐层输出所用的二维矩阵$\mathbf{W}$训练多组（具体训练多少组根据参数K而定），那么最终该层到下一层不再是传统的计算完后直接激活，而是从这K组输出值$Z_k$中选出值最大的一个作为下一隐层的输入，这就是Maxout的基本思想,论文中用以下公式进行表达：

$$h_i(x) = max_{j \in [1,k]}Z_{ij}$$

由上述阐述可知，Maxout的作用类似于我们激活函数(ReLU, Sigmoid, tanh等等)，但它又不同于传统的激活函数，因为他不是一个固定的激活函数，他是一个可学习的激活函数，因为我们的参数$\mathbf{W}$时可学习的，具体来说，他其实是一个分段线性函数(Piecewise Linear Function)。

<img src="http://img.blog.csdn.net/20160102203555432?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQv/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center" width="300">

**而任何一个凸函数，都可以由线性分段函数进行逼近近似。**其实我们可以把以前所学到的激活函数：relu、abs激活函数，看成是分成两段的线性函数，

<img src="http://img.blog.csdn.net/20160102210809727?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQv/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/Center">

因此，可以说Maxout的拟合能力非常强，它可以拟合任意的凸函数，**最直观的解释就是任意的凸函数都可以由分段线性函数以任意精度拟合，**而Maxout又是取k个“隐隐含层节点”的最大值，这些”隐隐含层"节点也是线性的，所以在不同的取值范围下，最大值也可以看做是分段线性的，其中分段的个数与K值的选取有关。

因此，Maxout可以理解为是一个函数逼近器，对于一个标准的MLP网络来说，如果隐藏层的神经元足够多，那么理论上我们是可以逼近任意的函数的。（**定理1：对于任意的一个连续分段线性函数$g(v)$，我们可以找到两个凸的分段线性函数$h1(v)、h2(v)$，使得这两个凸函数的差值为$g(v)$**）。

下图为传统CNN、Maxout Network以及NIN的卷积的区别：

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-5.png" width="600">

## NIN提出的前提

传统CNN的``data patch``与卷积核内积，之后送入激活层的结构，对`data patch`抽象能力不行，抽象能力指的是对同样的概念(比如两张图片都代表猫，但猫在两张图片中所处的位置和大小不同)，卷积层的输出值应该是不变的。

传统的`CNN`，就简单使用`data patch`与`filter`做内积，然后就求出来的值送入激活函数`ReLU`中。如果表示相同概念的`data patch`稍有不同的话，那么与`filter`做完内积的值也会变化（可能会和之前结果相差甚远），这显然不是我们想要的结果，我们想要**不变性**。所以作者提出了`NIN`，用来提高卷积层的抽象能力。

## 网络结构

作者对传统的CNN结构做了两部分的修改：
* 将原始卷积神经网络中的单层感知机（也可以说是线性卷积层）替换成了多层感知机，结构如下图：

<img src="https://upload-images.jianshu.io/upload_images/274952-72e1d65915ed03d4.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/685" width="500">

* 放弃传统`CNN`的全连接层，在最后一层的`feature map`使用`spatial pooling`（Global Average Pooling）作为`NIN`的分类的结果。已经证实，全连接层非常容易导致过拟合，极度依赖`dropout`。所以去除全连接层还是有好处的。作者还从另一个角度解释`spatial pooling`：迫使网络的最后一层学习到与类别相关的特征。(<font color="green">作者提出，传统的CNN最后采用全连接做分类，这样有缺点，一是容易导致过拟合，二是，之所以深度学习不能被解释，就是因为全连接层的存在，因此作者提出采用平均池化层来做代替，并且最后加上softmax。这样有效的原因是，这种做法，把特征图和分类直接连了起来。</font>)

<img src="https://upload-images.jianshu.io/upload_images/274952-62b244a596310e4e.jpg?imageMogr2/auto-orient/strip%7CimageView2/2/w/700" width="600">

传统的CNN中，需要用较多的卷积核来兼顾各种可能的隐藏概念，这会加重网络的负担，作者认为应该在CNN中的下一层组合当前层的特征之前，提取出更高级别的特征，然后在进行组合。

<img src="http://olvorb0zr.bkt.clouddn.com/20171027%E5%85%A8%E5%B1%80%E5%9D%87%E5%80%BC%E6%B1%A0%E5%8C%96.JPG" width="550">

NIN的笔记主要参考

1.[这篇博文](https://www.jianshu.com/p/96791a306ea5)

2.[NIN笔记](http://chenpeng.online/2017/10/27/Network-in-Network-%E7%AC%94%E8%AE%B0/)

3.[卷积神经网络中用1*1 卷积有什么作用或者好处呢？](https://www.zhihu.com/question/56024942)

## 为什么mlpconv里面要有两个$1×1$卷积

传统的卷积层可以看成是一种的广义的线性模型（Generalized Linear Model），即$f_{i,j,k}=max(W_k^Tx_{i,j}, 0)$。

因此，如果提取到的特征是线性可分的话，传统的CNN层对特征的抽象其实是足够的，但是，好的抽象一般需要对输入的数据做高度的非线性变换，因此以往的CNN通常有两个做法：
* 在同一层中使用多个通道来覆盖同一个input data patch特征的所有variations，其相当于增加信息冗余，来弥补线性变换的不足。
* 多个CNN层的堆叠来获得前层特征的更高抽象，同时可以获取更大的感受野，这种操作会使得参数和计算量增加的都太快。

因此，左右利用$1 \times 1$的Kernel来模拟这种操作，即实现了上述两个目的，又减少了参数的数量和计算量。mlpconv中的第一个$1 \times 1$的CNN层可以看成是对前一层素有feature map信息进行线性组合，因为是$1 \times 1$的kernel，所以可以看成是一种变相的全连接层，在使用ReLU进行非线性变换，就实现了一次特征的整合和非线性变换抽象。但是这和传统的CNN层并没有什么区别，也没有实现作者所说的要对input data patch做高度的非线性变换的目的，因此作者又加了一个同样的$1 \times 1$的CNN层，这样就实现了对输入的数据进行高度的非线性变换的目的，加入这两个”全连接“，确实在不怎么增加参数和计算量的同时，实现了对特征的高度整合和抽象，因此也会获得更好的效果。

但其实作者最大的贡献还是在最后用”Global Average Pooling“代替了全连接层，在降低参数的同时，也保留了前层所学习到的所有特征，使得模型的过拟合和训练都得到了改善。

In [1]:
import mxnet as mx
import numpy as np

from mxnet import nd
from mxnet import gluon
from mxnet import autograd
mx.random.seed(1)

import utils
ctx = mx.gpu()

In [2]:
def mlpconv(channels, kernel_size, padding, strides=1, max_pooling=True):
    net = gluon.nn.Sequential()
    net.add(gluon.nn.Conv2D(channels=channels, kernel_size=kernel_size, strides=strides, 
                            padding=padding, activation="relu"))
    net.add(gluon.nn.Conv2D(channels=channels, kernel_size=1, activation='relu'))
    net.add(gluon.nn.Conv2D(channels=channels, kernel_size=1, activation='relu'))
    if max_pooling:
        net.add(gluon.nn.MaxPool2D(pool_size=3, strides=2))
    return net

In [3]:
blk = mlpconv(64, 3, 0)
blk.initialize(ctx=ctx)
X = nd.random.uniform(shape=(32,5,16,16), ctx=ctx)
blk(X).shape

(32, 64, 6, 6)

In [4]:
blk.collect_params()

sequential0_ (
  Parameter conv0_weight (shape=(64, 5, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter conv0_bias (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter conv1_weight (shape=(64, 64, 1, 1), dtype=<class 'numpy.float32'>)
  Parameter conv1_bias (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter conv2_weight (shape=(64, 64, 1, 1), dtype=<class 'numpy.float32'>)
  Parameter conv2_bias (shape=(64,), dtype=<class 'numpy.float32'>)
)

## NIN网络参数

kernel: $11 \times 11$, channels: 96, padding: 0, strides: 4

kernel: $5 \times 5$, channels: 256, padding: 2, strides: 1

kernel: $3 \times 3$, channels: 384, padding: 1, strides: 1

除了使用了$1 \times 1$卷积外，NiN在最后不是使用全连接，而是使用通道数为输出类别个数的mlpconv，外接一个平均池化层来将每个通道里的数值平均成一个标量.

In [5]:
def get_nin():
    net = gluon.nn.Sequential()
    with net.name_scope():
        net.add(mlpconv(96, kernel_size=11, padding=0, strides=4))
        net.add(mlpconv(256, kernel_size=5, padding=2))
        net.add(mlpconv(384, kernel_size=3, padding=1))
        # 使用通道数等于输出类别个数的mlpconv
        net.add(mlpconv(10, kernel_size=3, padding=1, max_pooling=False))
        # 外加一个AvgPooling2D来将每个通道里的数值平均为一个标量
        net.add(gluon.nn.AvgPool2D(pool_size=5))
        net.add(gluon.nn.Flatten())
    return net

In [6]:
nin = get_nin()
nin.initialize(ctx=ctx)

In [7]:
Y = nd.random.uniform(shape=(64,3,224,224), ctx=ctx)
nin(Y).shape

(64, 10)

## 训练

In [8]:
from time import time

batch_size = 64
train_data, test_data = utils.load_dataset(batch_size, resize=224, data_type='cifar10')

nin = get_nin()
nin.collect_params().initialize(mx.init.Xavier(), ctx=ctx, force_reinit=True)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(nin.collect_params(), 'sgd', {'learning_rate': 0.1})

epochs = 20

niter = 0
moving_loss = 0.0
smoothing_constant = 0.9

from time import time
for epoch in range(epochs):
    start = time()
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = nin(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        
        niter += 1
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = smoothing_constant * moving_loss + (1-smoothing_constant) * curr_loss
        estimated_loss = moving_loss / (1 - smoothing_constant**niter)
    
    train_acc = utils.evaluate_accuracy_gluon(train_data, nin, ctx)
    test_acc = utils.evaluate_accuracy_gluon(test_data, nin, ctx)
    print("Epoch %d, Moving Train Avg loss %.5f, Train acc %.5f, Test acc %.5f, Time consume %.5f s."
         % (epoch, estimated_loss, train_acc, test_acc, time() - start))

Epoch 0, Moving Train Avg loss 2.27351, Train acc 0.14322, Test acc 0.14100, Time consume 370.98654 s.
Epoch 1, Moving Train Avg loss 2.08578, Train acc 0.24340, Test acc 0.25160, Time consume 385.06472 s.
Epoch 2, Moving Train Avg loss 1.92863, Train acc 0.30864, Test acc 0.31210, Time consume 294.13755 s.
Epoch 3, Moving Train Avg loss 1.80213, Train acc 0.39296, Test acc 0.39190, Time consume 147.64240 s.
Epoch 4, Moving Train Avg loss 1.52093, Train acc 0.41786, Test acc 0.41900, Time consume 147.76886 s.
Epoch 5, Moving Train Avg loss 1.41106, Train acc 0.48736, Test acc 0.48130, Time consume 156.67500 s.
Epoch 6, Moving Train Avg loss 1.35206, Train acc 0.54106, Test acc 0.52390, Time consume 148.68405 s.
Epoch 7, Moving Train Avg loss 1.22092, Train acc 0.57830, Test acc 0.56380, Time consume 147.20557 s.
Epoch 8, Moving Train Avg loss 1.07990, Train acc 0.63698, Test acc 0.61550, Time consume 149.97740 s.
Epoch 9, Moving Train Avg loss 1.12105, Train acc 0.64306, Test acc 0.619